# VacationPy Challenge
----

In [2]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
CityWeatherPath = "./CitiesWeather.csv"
CityWeather = pd.read_csv(CityWeatherPath)
CityWeather

,City,Latitude,Longitude,MaxTemp(F),Humidity(%),Cloudiness(%),WindSpeed(mph)
0,Vardø,70.3705,31.1107,57.24,71,38,8.88
1,Alberdi,-32.8899,-60.6906,79.92,46,0,3.44
2,Albany,42.6001,-73.9662,84.15,75,99,2.86
3,Vaini,-21.2000,-175.2000,71.76,73,75,8.05
4,Tatarsk,55.2190,75.9828,60.30,90,100,8.30
...,...,...,...,...,...,...,...
539,Turukhansk,65.8167,87.9833,58.91,71,36,2.39
540,Progreso,21.2833,-89.6667,93.31,69,21,18.03
541,Narrabri,-30.3167,149.7833,38.52,91,0,1.01
542,Beyneu,45.3167,55.2000,87.08,17,0,7.81


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
CityWeather["Humidity(%)"] = CityWeather["Humidity(%)"].astype(float)
CityWeather.dtypes

City               object
Latitude          float64
Longitude         float64
MaxTemp(F)        float64
Humidity(%)       float64
Cloudiness(%)       int64
WindSpeed(mph)    float64
dtype: object

In [7]:
# configure gmaps
gmaps.configure(api_key=g_key)

# store the data into variables
locations = CityWeather[["Latitude","Longitude"]]
humidity = CityWeather["Humidity(%)"].astype(float)
maxhumidity = CityWeather["Humidity(%)"].max()

# create the map and add heat layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                dissipating = True, max_intensity = maxhumidity)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
IdealWeather = CityWeather.loc[(CityWeather["MaxTemp(F)"]<75)&(CityWeather["MaxTemp(F)"]>65)&
                               (CityWeather["Humidity(%)"]<60)&(CityWeather["Cloudiness(%)"]>0)&
                               (CityWeather["Cloudiness(%)"]<90),:]
IdealWeather

,City,Latitude,Longitude,MaxTemp(F),Humidity(%),Cloudiness(%),WindSpeed(mph)
42,Huancabamba,-5.2386,-79.4506,71.13,49.0,11,2.15
70,Whitehorse,60.7161,-135.0538,66.02,58.0,71,2.10
101,Itaqui,-29.1253,-56.5531,74.32,54.0,2,9.24
163,Haines Junction,60.7522,-137.5108,72.16,55.0,61,3.98
213,Balkhash,46.8440,74.9804,73.38,46.0,9,6.22
225,Kamina,-8.7386,24.9906,66.31,26.0,82,4.00
271,Englehart,47.8169,-79.8664,71.67,39.0,58,4.43
283,Chapais,49.7834,-74.8492,66.07,34.0,3,7.76
408,Santa Luzia,-19.7697,-43.8514,67.10,56.0,40,8.05
441,Tinde,-3.8833,33.2000,73.89,38.0,40,5.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = pd.DataFrame(IdealWeather)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Latitude,Longitude,MaxTemp(F),Humidity(%),Cloudiness(%),WindSpeed(mph),Hotel Name
42,Huancabamba,-5.2386,-79.4506,71.13,49.0,11,2.15,
70,Whitehorse,60.7161,-135.0538,66.02,58.0,71,2.10,
101,Itaqui,-29.1253,-56.5531,74.32,54.0,2,9.24,
163,Haines Junction,60.7522,-137.5108,72.16,55.0,61,3.98,
213,Balkhash,46.8440,74.9804,73.38,46.0,9,6.22,
225,Kamina,-8.7386,24.9906,66.31,26.0,82,4.00,
271,Englehart,47.8169,-79.8664,71.67,39.0,58,4.43,
283,Chapais,49.7834,-74.8492,66.07,34.0,3,7.76,
408,Santa Luzia,-19.7697,-43.8514,67.10,56.0,40,8.05,
441,Tinde,-3.8833,33.2000,73.89,38.0,40,5.50,


In [42]:
# find the closest hotel within 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "hotel"
radius = 5000 

params = {
    "type":target_search,
    "radius":radius,
    "key":g_key
}


In [43]:
for index, row in hotel_df.iterrows():
    city = row["City"]
    params['keyword'] = city
    
    print(f'Retrieving results for index {index}: {city}')
    response = requests.get(base_url,params=params).json()
    
    results = response['results']
    
    try:
        print(f'Closest hotel is {results[0]['name']}.')
        
        hotel_df.loc[index,'Hotel Name']=results[0]['name']
    
    except (KeyError,IndexError):
        print("Missing field/result...skipping")
    
    print("----------------------------------------")

SyntaxError: invalid syntax (<ipython-input-43-94717455f306>, line 11)

In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
